In [1]:
import torch

In [8]:
def create_rand(n1, n2=1, device='cuda'):
    if n2==1:
        return torch.rand((n1), device=device)
    else:
        return torch.rand((n1, n2), device=device)

In [7]:
def create_int(n1, n2=1, device='cuda'):
    if n2==1:
        return torch.randint(1, 54999, (n1,), device=device)
    else:
        return torch.randint(1, 54999, (n1, n2), device=device)

In [58]:
a=torch.randint(1, 54999, (500,))
b=torch.rand((500,))
a1=torch.randint(1, 54999, (500,))
b1=torch.rand((500,))

def f(a, b, a1, b1, device='cuda'):
    a=a.to(device)
    b=b.to(device)
    a1=a1.to(device)
    b1=b1.to(device)
    v=torch.zeros((55000), device=device)
    v[a] = b
    v1=torch.zeros((55000), device=device)
    v1[a1]=b1
    return torch.inner(v, v1)
f(a, b, a1, b1)
# timeit f(a, b, a1, b1)

def f1(a, b, a1, b1):
    k = {}
    for ia, ib in zip(a, b):
        k[int(ia)] = float(ib)
    v=0
    for ia, ib in zip(a1, b1):
        v += k.get(int(ia), 0)*float(ib)
    return v

In [59]:
f(a, b, a1, b1)

tensor(1.1938, device='cuda:0')

In [63]:
def f100(a, b, a1, b1, device='cuda'):
    a=a.to(device)
    b=b.to(device)
    a1=a1.to(device)
    b1=b1.to(device)
    for i in range(100):
        v=torch.zeros((55000), device=device)
        v[a] = b
        v1=torch.zeros((55000), device=device)
        v1[a1]=b1
        r = torch.dot(v, v1)
    return r

In [49]:
def f2(a, b, a1, b1, device='cuda'):
    v=torch.zeros((100, 55000), device=device)
    for i in range(a.size()[0]):
        v[i][a[i]] = b[i]
    v1=torch.zeros((100, 55000), device=device)
    for i in range(a1.size()[0]):
        v1[i][a[i]] = b[i]
    return torch.matmul(v.reshape(100,1,55000), v1.reshape(100,55000,1)).squeeze(1)


In [65]:
%timeit f100(a, b, a1, b1, 'cpu')

6.14 ms ± 173 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%timeit f1(a, b, a1, b1)

1.46 ms ± 4.31 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
109 μs ± 909 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [66]:
%timeit f(a, b, a1, b1, 'cpu')

112 μs ± 847 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
aa=create_int(100, 500)
aa1=create_int(100, 500)
bb=create_rand(100, 500)
bb1=create_rand(100, 500)

In [67]:
%timeit f2(aa, bb, aa1, bb1)

1.65 ms ± 6.98 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
